So last time, the kernel was dying as we tried to change the array to a pandas dataframe... Now we will transform the pictures to a smaller size using cv2, inspired by the YOLO implementation showed by the course. We will construct the labels as we want are 1 and 0 depending on what we will train. The objective beeing to prepare a parallel training and testing for a team of 3 small, ANN. 

In [1]:
!pip install opencv-python


In [2]:
# The code was removed by Watson Studio for sharing.

This last bit of code I got it from Stack Overflow https://stackoverflow.com/questions/48274094/watson-data-platform-how-to-unzip-the-zip-file-in-the-data-assets <br />
I'll add here use cv2 to try to reduce the image size to 415 by 415, and at it is an iterative process I'll create as well the lables, as 1,2,3. This will be specially usefull when I'll try to pass it to a one-hot encoded format. 

In [3]:
import cv2
from io import BytesIO
import zipfile
import matplotlib.pyplot as plt
import numpy as np
images = []
labels=[]
streambodies = [streaming_body_1,streaming_body_2,streaming_body_3]
i=0
for streambody in streambodies:
    i=i+1
    zip_ref = zipfile.ZipFile(BytesIO(streambody.read()), 'r')
    my_list_file = zip_ref.namelist()
    for filepath in my_list_file:
        if filepath[-1] == 'g':
            image = cv2.imread(zip_ref.extract(filepath),0)
            
            #Resizing
            image = cv2.resize(image, (416, 416))
            #Normalization
            image = image / 255.
            
            image = image.flatten()
            labels.append(i)
            images.append(image)
    zip_ref.close()
images = np.array(images)
labels = np.array(labels)
print(np.shape(images))

(852, 173056)


In [4]:
df = pd.DataFrame(images)
df['labels'] = labels
print(np.shape(df))
df.head()


(852, 173057)


,0,1,2,3,4,5,6,7,8,9,...,173047,173048,173049,173050,173051,173052,173053,173054,173055,labels
0,0.937255,0.941176,0.941176,0.941176,0.941176,0.941176,0.937255,0.941176,0.937255,0.941176,...,0.941176,0.945098,0.945098,0.941176,0.937255,0.937255,0.937255,0.941176,0.941176,1
1,0.933333,0.933333,0.937255,0.937255,0.933333,0.933333,0.933333,0.933333,0.929412,0.933333,...,0.933333,0.937255,0.933333,0.937255,0.933333,0.929412,0.929412,0.929412,0.933333,1
2,0.917647,0.917647,0.917647,0.917647,0.917647,0.921569,0.921569,0.917647,0.917647,0.917647,...,0.917647,0.925490,0.921569,0.921569,0.917647,0.917647,0.917647,0.921569,0.917647,1
3,0.941176,0.945098,0.945098,0.945098,0.941176,0.937255,0.945098,0.941176,0.941176,0.941176,...,0.941176,0.945098,0.941176,0.941176,0.941176,0.941176,0.941176,0.945098,0.945098,1
4,0.921569,0.921569,0.921569,0.921569,0.921569,0.917647,0.921569,0.917647,0.917647,0.921569,...,0.921569,0.921569,0.917647,0.917647,0.917647,0.913725,0.913725,0.921569,0.921569,1


This looks preeeetty good. Let's create from the label column a one hot encoder to be able to train seperatly each ANN. 

In [5]:
df = pd.concat([df,pd.get_dummies(df['labels'], prefix='label')],axis=1)
df.drop(['labels'],axis=1, inplace=True)

print(np.shape(df))

df.head()

(852, 173059)


,0,1,2,3,4,5,6,7,8,9,...,173049,173050,173051,173052,173053,173054,173055,label_1,label_2,label_3
0,0.937255,0.941176,0.941176,0.941176,0.941176,0.941176,0.937255,0.941176,0.937255,0.941176,...,0.945098,0.941176,0.937255,0.937255,0.937255,0.941176,0.941176,1,0,0
1,0.933333,0.933333,0.937255,0.937255,0.933333,0.933333,0.933333,0.933333,0.929412,0.933333,...,0.933333,0.937255,0.933333,0.929412,0.929412,0.929412,0.933333,1,0,0
2,0.917647,0.917647,0.917647,0.917647,0.917647,0.921569,0.921569,0.917647,0.917647,0.917647,...,0.921569,0.921569,0.917647,0.917647,0.917647,0.921569,0.917647,1,0,0
3,0.941176,0.945098,0.945098,0.945098,0.941176,0.937255,0.945098,0.941176,0.941176,0.941176,...,0.941176,0.941176,0.941176,0.941176,0.941176,0.945098,0.945098,1,0,0
4,0.921569,0.921569,0.921569,0.921569,0.921569,0.917647,0.921569,0.917647,0.917647,0.921569,...,0.917647,0.917647,0.917647,0.913725,0.913725,0.921569,0.921569,1,0,0


Looks like we are ready to train to identify each object! Let's go!